In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key='your groq api key',
    model_name = "llama-3.1-70b-versatile",
    temperature=0
)

In [ ]:
response = llm.invoke('who is the ceo of worldquant')
print (response)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-40807?from=job%20search%20funnel")

In [ ]:
page_data = loader.load().pop().page_content
print(page_data)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

In [ ]:
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

json_res = json_parser.parse(res.content)

In [ ]:
json_res

Prepare Chroma DB

In [ ]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

In [ ]:
job = json_res
job['skills']